In [1]:
import tensorflow as tf
import tflearn
import numpy as np
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from random import shuffle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# Defining constants
#ver 1 = 10 epochs, 32 filters, 256 fc neurons
# learning rate of 1 = 1e-3
#Learning rate of 0.01 = 1e-2
IMG_SIZE = 54 # 50 x 50 image, for example
TIME_LEN = 10 # 5 time points, for example
LR = 1e-3 # learning rate, 1e-3 for example
CONV_LAYERS = 1
POOL_LAYERS = 1
NO_CHANNELS = 1 # how many input streams i.e. a network with 3 channels could have RGB, depth, and body skeleton streams
VERSION = '1' # to keep track of changes in a particular model
NO_SIGNS = 3 # number of signs to learn - i.e. the alphabet
N_EPOCH = 50 # number of epochs
MODEL_NAME = '3dconv-{}--{}--{}--{}--{}-.model'.format(LR, CONV_LAYERS, POOL_LAYERS, NO_CHANNELS, VERSION)

In [3]:
conv_net = input_data(shape = [None, TIME_LEN, IMG_SIZE, IMG_SIZE, NO_CHANNELS], name = 'input')

conv_net = conv_3d(conv_net, 32, 3, strides = 1, padding = 'same', activation = 'relu')

conv_net = max_pool_3d(conv_net, 2, strides = 2)

#conv_net = conv_3d(conv_net, 32, 3, strides = 1, padding = 'same', activation = 'relu')

#conv_net = max_pool_3d(conv_net, 2, strides = 2)

#conv_net = conv_3d(conv_net, 32, 3, strides = 1, padding = 'same', activation = 'relu')

conv_net = fully_connected(conv_net, 256, activation = 'relu')

conv_net = dropout(conv_net, 0.8)

conv_net = fully_connected(conv_net, NO_SIGNS, activation = 'softmax')

conv_net = regression(conv_net, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
train_data = list(np.load("trainingData.npy"))
#train_data_extra = np.load("trainingDataRotated.npy")
shuffle(train_data)
#shuffle(train_data_extra)
#train_data_all = list(np.concatenate((train_data, train_data_extra), axis = 0))
#shuffle(train_data_all)

X = np.array([i[0] for i in train_data]).reshape([-1, 10, 54, 54, 1])
Y = [i[1] for i in train_data]

In [5]:
model = tflearn.DNN(conv_net, tensorboard_dir = 'log', checkpoint_path = '3dconv-0.001-1-1-1-1', best_checkpoint_path = '3dconv-0.001-1-1-1-1-best',
                    max_checkpoints = 5, tensorboard_verbose = 1)

In [6]:
model.fit(X, Y, n_epoch = N_EPOCH, validation_set = 0.1, show_metric = True, run_id = MODEL_NAME, snapshot_step = 10) 


Training Step: 18  | total loss: 13.29070 | time: 128.513s
| Adam | epoch: 002 | loss: 13.29070 - acc: 0.4228 -- iter: 320/805


KeyboardInterrupt: 

In [ ]:
model.save('3dconv-0.001-1-1-1-1.tflearn')

In [ ]:
test_data = np.load('testingData.npy')
X_test = np.array([test_data[i] for i in range(0, len(test_data))]).reshape([-1, 10, 54, 54, 1])
test2 = X[300]
testy2 = Y[300]

In [ ]:
model.predict({'input':[test2]})

In [ ]:
testy2